In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
data=pd.read_csv('GEFCom2014N.csv').dropna()
data['load']=data['load']/10000
data['T']=data['T']/100
# Convert timestamp to datetime format
data['Datetime'] = data['Datetime'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

# Set the datetime as index
data.set_index('Datetime', inplace=True)

# Resample the data to hourly frequency
data = data.resample('H').mean()

# Create lag features
for i in range(1, 25):
    data['lag_{}'.format(i)] = data['load'].shift(i)

# Create rolling mean and standard deviation features
data['rolling_mean'] = data['load'].rolling(window=24).mean()
data['rolling_std'] = data['load'].rolling(window=24).std()

# Create weekday and hour features
data['weekday'] = data.index.weekday
data['hour'] = data.index.hour

# Remove missing values
data.dropna(inplace=True)

# Split the data into train and test sets
train_size = int(len(data) * 0.8)
train_data = data[:train_size]
test_data = data[train_size:]

In [3]:
#GoogleNet Lstm
import numpy as np
import tensorflow as tf
from tensorflow.keras import models, layers

from sklearn.metrics import mean_squared_error

# Prepare the train and test data
X_train, y_train = train_data.drop('load', axis=1), train_data['load']
X_test, y_test = test_data.drop('load', axis=1), test_data['load']

# Reshape the data for CNN-LSTM
X_train = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1)

# Build the GoogleNet-LSTM model
def inception_module(x, filters):
    branch1x1 = layers.Conv1D(filters[0], 1, activation='relu', padding='same')(x)
    
    branch3x3 = layers.Conv1D(filters[1], 1, activation='relu', padding='same')(x)
    branch3x3 = layers.Conv1D(filters[2], 3, activation='relu', padding='same')(branch3x3)
    
    branch5x5 = layers.Conv1D(filters[3], 1, activation='relu', padding='same')(x)
    branch5x5 = layers.Conv1D(filters[4], 5, activation='relu', padding='same')(branch5x5)
    
    branch_pool = layers.MaxPooling1D(3, strides=1, padding='same')(x)
    branch_pool = layers.Conv1D(filters[5], 1, activation='relu', padding='same')(branch_pool)
    
    output = layers.concatenate([branch1x1, branch3x3, branch5x5, branch_pool], axis=-1)
    return output

input_shape = (X_train.shape[1], X_train.shape[2])  # Adjust input shape as per your requirements

input_layer = layers.Input(shape=input_shape)
x = layers.Conv1D(6, kernel_size=5, activation='relu', padding='same')(input_layer)
x = layers.MaxPooling1D(pool_size=2)(x)
x = layers.Conv1D(16, kernel_size=5, activation='relu', padding='same')(x)
x = layers.MaxPooling1D(pool_size=2)(x)

# Replace fully connected layers with Inception modules
x = inception_module(x, [6, 6, 8, 2, 4, 4])
x = inception_module(x, [16, 8, 12, 4, 8, 8])

x = layers.LSTM(84, return_sequences=True)(x)
x = layers.Flatten()(x)
x = layers.Dense(1, activation='linear')(x)

model = models.Model(inputs=input_layer, outputs=x)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32)

# Evaluate the model
loss = model.evaluate(X_test, y_test)
print('Test loss:', loss)

# Make predictions
predictions = model.predict(X_test)

ypred = predictions.flatten()
y_test=test_data['load']
y_glstm=ypred



Epoch 1/100
1972/1972 [==============================] - 27s 11ms/step - loss: 5.5740e-04
Epoch 2/100
1972/1972 [==============================] - 22s 11ms/step - loss: 6.3911e-05
Epoch 3/100
1972/1972 [==============================] - 22s 11ms/step - loss: 5.3679e-05
Epoch 4/100
1972/1972 [==============================] - 22s 11ms/step - loss: 4.6328e-05
Epoch 5/100
1972/1972 [==============================] - 22s 11ms/step - loss: 4.0006e-05
Epoch 6/100
1972/1972 [==============================] - 22s 11ms/step - loss: 3.7197e-05
Epoch 7/100
1972/1972 [==============================] - 22s 11ms/step - loss: 3.3860e-05
Epoch 8/100
1972/1972 [==============================] - 21s 11ms/step - loss: 3.2355e-05
Epoch 9/100
1972/1972 [==============================] - 351s 178ms/step - loss: 2.8900e-05
Epoch 10/100
1972/1972 [==============================] - 21s 11ms/step - loss: 2.7862e-05
Epoch 11/100
1972/1972 [==============================] - 22s 11ms/step - loss: 2.7005e-05
Epoch 

1972/1972 [==============================] - 22s 11ms/step - loss: 1.0135e-05
Epoch 92/100
1972/1972 [==============================] - 23s 12ms/step - loss: 1.0044e-05
Epoch 93/100
1972/1972 [==============================] - 23s 12ms/step - loss: 9.7141e-06
Epoch 94/100
1972/1972 [==============================] - 23s 12ms/step - loss: 1.0319e-05
Epoch 95/100
1972/1972 [==============================] - 21s 11ms/step - loss: 9.6583e-06
Epoch 96/100
1972/1972 [==============================] - 23s 11ms/step - loss: 1.0139e-05
Epoch 97/100
1972/1972 [==============================] - 21s 10ms/step - loss: 9.8696e-06
Epoch 98/100
1972/1972 [==============================] - 19s 10ms/step - loss: 9.8735e-06
Epoch 99/100
1972/1972 [==============================] - 20s 10ms/step - loss: 9.9230e-06
Epoch 100/100
493/493 [==============================] - 3s 4ms/step - loss: 9.5800e-06
Test loss: 9.579974175721873e-06
493/493 [==============================] - 3s 4ms/step


In [4]:
# For MAE
mae = np.mean(np.abs(ypred - y_test))
print('MAE = {:.2f} (%)'.format(mae))

# For MAPE
mape = np.mean(np.abs((y_test - ypred) / y_test)) * 100
print('MAPE = {:.2f} (%)'.format(mape))

# For RMSE
mse = mean_squared_error(y_test, ypred) 
rmse = np.sqrt(mse) 
print('RMSE = {:.2f} (%)'.format(rmse))

MAE = 0.00 (%)
MAPE = 0.68 (%)
RMSE = 0.00 (%)
